In [138]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing the libraries**


In [140]:
import cudf as pd
import cupy as cp
import cuml
from cuml.linear_model import LogisticRegression
from cuml.linear_model import LogisticRegression
from cuml.multiclass import MulticlassClassifier
from cuml.multiclass import OneVsRestClassifier





#Text Preprocessing Libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import re
import string

#Deep Learning Libraries
import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Embedding
from keras.layers import Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

In [141]:
train_df=pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip',sep='\t')
test_df=pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip',sep='\t')

In [142]:
train_df.head()

In [143]:
train_df.info()

In [144]:
train_df.shape

**Data Preprocessing**

In [145]:
train_df.isnull().sum()




In [146]:
test_df.isnull().sum()

In [147]:
train_df.isnull().any().any()

In [148]:
test_df.isnull().any().any()

In [149]:
train_df['Sentiment'].value_counts()

The sentiment labels are:

0 : only negative

1 : majorly negative

2 : neutral

3 : majorly positive

4 : only positive


**Text Preprocessing**
Steps-

1) Remove Punctuations

2) Remove HTML Tags

3) Convert text to lowercase

4) Remove all special characters

5) Remove stopwords

6) Perform Stemming

7) Join back after stemming


In [150]:
test=test_df.to_pandas()
train=train_df.to_pandas()

In [151]:
import string
import re
string.punctuation

In [152]:
#removing punctuations
train['Phrase']=train['Phrase'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)).lower())
#removing punctuations
test['Phrase']=test['Phrase'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)).lower())

In [153]:
#remove html tags
def clean_html(text):
    clean=re.compile('<.*?>')
    return re.sub(clean,'',text)

In [154]:
#convert text to lowercase
def convert_lower(text):
    return text.lower()

In [155]:
#remove special characters
def remove_special(text):
    x=''
    
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x

In [156]:
train['Phrase']=train['Phrase'].apply(convert_lower)
test['Phrase']=test['Phrase'].apply(convert_lower)

In [157]:
train['Phrase']=train['Phrase'].apply(clean_html)
test['Phrase']=test['Phrase'].apply(clean_html)

In [158]:
train['Phrase']=train['Phrase'].apply(remove_special)
test['Phrase']=test['Phrase'].apply(remove_special)

In [159]:
#stopwords.words('english')

In [160]:
#remove stopwords
def remove_stopwords(text):
    x=[]
    for i in text.split():
        
        if i not in stopwords.words('english'):
            x.append(i)
    
    y=x[:]
    x.clear()
    return y

In [161]:
train['Phrase']=train['Phrase'].apply(remove_stopwords)
test['Phrase']=test['Phrase'].apply(remove_stopwords)

In [162]:
ps=PorterStemmer()

In [163]:
y=[]
#stem the words
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z

In [164]:
train['Phrase']=train['Phrase'].apply(stem_words)
test['Phrase']=test['Phrase'].apply(stem_words)

In [165]:
#join the stem words together
def join_back(list_input):
    return " ".join(list_input)

In [166]:
train['Phrase']=train['Phrase'].apply(join_back)
test['Phrase']=test['Phrase'].apply(join_back)

In [167]:
X_train =train['Phrase']
y_train = train['Sentiment']
tokenize = Tokenizer()
tokenize.fit_on_texts(X_train.values)

In [168]:
X_test = test['Phrase']
X_train = tokenize.texts_to_sequences(X_train)
X_test = tokenize.texts_to_sequences(X_test)

In [169]:
#add padding using pre-method
max_sequence_len = max([len(s.split()) for s in train['Phrase']])
X_train = pad_sequences(X_train, max_sequence_len,padding='pre')
X_test = pad_sequences(X_test, max_sequence_len,padding='pre')

In [170]:
X_train

In [171]:
print(X_train.shape)
print(X_test.shape)

**Implementation using Machine Learning Models-using Sklearn.**

The sklearn library contains a lot of efficient tools for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction.

In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score

In [173]:
lr = LogisticRegression(random_state=0)

In [174]:
lr.fit(X_train,y_train)

In [175]:
pred_lr=lr.predict(X_test)

In [176]:
pred_lr

In [177]:
np.unique(pred_lr)

**Machine Learning Models using Cuml**

Logistic Regression

In [178]:
lg = OneVsRestClassifier(LogisticRegression())

In [179]:
lg.fit(X_train.astype('float32'),y_train.astype('float32'))

In [180]:
y_pred_lg = lg.predict(X_test.astype('float32'))

In [181]:
y_pred_lg=y_pred_lg.astype('int32')

In [182]:
y_pred_lg

In [183]:
cp.unique(y_pred_lg)

In [184]:
#Accurcy score
cu_score = cuml.metrics.accuracy_score(y_train, y_pred_lg )

In [185]:
cu_score


**Deep Learning Models**


In [186]:
#LSTM Model
embedding_dimension = 100
input_val = len(tokenize.word_index)+1
model_lstm = Sequential()
model_lstm.add(Embedding(input_val, embedding_dimension, input_length=max_sequence_len))
model_lstm.add(LSTM(units=256, dropout=0.3, recurrent_dropout=0.2 , return_sequences=True))
model_lstm.add(LSTM(units=256, dropout=0.3, recurrent_dropout=0.2 ))
model_lstm.add(Dense(5, activation='softmax'))# 5 as we are required to predict 5 labels-0,1,2,3,4

In [187]:
model_lstm.summary()

In [188]:
early_stopping = EarlyStopping(min_delta = 0.001,
                               mode = 'max',
                               monitor = 'val_acc',
                               patience = 2)
callback = [early_stopping]

In [189]:
#Compiling the model
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [190]:
#Fit the model
history_lstm=model_lstm.fit(X_train, y_train,batch_size=128, epochs=10, verbose=1,callbacks=callback)

In [191]:
#Saving the model
model_lstm.save('LSTM_Model.h5')

In [192]:
loss_train = history_lstm.history['loss']
epochs = range(1,11)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [193]:
accuracy_train = history_lstm.history['accuracy']
epochs = range(1,11)
plt.plot(epochs, accuracy_train, 'g', label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [194]:
#Predictions on the model
predict_x=model_lstm.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

In [195]:
classes_x